<a href="https://colab.research.google.com/github/JiNYouNG2222/pattern-recognition/blob/main/final_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from collections import OrderedDict
from IPython.display import clear_output
from itertools import product

'''
learning_rate = [0.001, 0.0001]
batch_size = [16, 32, 64, 128]
num_classes = 26
epochs = 30
drop_out = [0.1, 0.3, 0.5]
'''


learning_rates = [0.001, 0.0001]
batch_sizes = [16, 32, 64, 128]
drop_outs = [0.1, 0.3, 0.5]
features = [1025, 2048, 1000, 512, 256]
num_classes = 26
epochs = 30

for learning_rate, batch_size, drop_out, feature in product(learning_rates, batch_sizes, drop_outs, features):
    # 디바이스 설정 (CUDA 사용 가능 시 GPU 사용)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # 데이터셋 및 데이터 로더 설정
    transform = transforms.Compose([
        transforms.ToTensor()  # 이미지를 텐서로 변환
    ])

    train_set = torchvision.datasets.EMNIST(
        root='./data/EMNIST',
        split='letters',
        train=True,
        download=True,
        transform=transform
    )
    test_set = torchvision.datasets.EMNIST(
        root='./data/EMNIST',
        split='letters',
        train=False,
        download=True,
        transform=transform
    )

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)


    #####
    # 신경망 모델 정의
    class NeuralNet(nn.Module):
        def __init__(self):
            super().__init__()

            # 합성곱 계층 정의
            self.layer1 = nn.Sequential(
                nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(16),  # 배치 정규화
                nn.ReLU(),
                nn.MaxPool2d(2)
            )
            self.layer2 = nn.Sequential(
                nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(),
                nn.MaxPool2d(2)
            )
            self.layer3 = nn.Sequential(
                nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(2)
            )
            self.layer4 = nn.Sequential(
                nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU()
            )
            self.dropout = nn.Dropout(p=drop_out)
            self.fc1 = nn.Linear(128 * 3 * 3, feature)
            self.fc2 = nn.Linear(feature, num_classes)

        def forward(self, x):
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = x.view(x.size(0), -1)  # Flatten
            x = self.dropout(x)
            x = self.fc1(x)
            x = self.fc2(x)
            return x

    #####

    # 모델 초기화, 손실 함수 및 최적화 기법 설정
    net = NeuralNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)

    # 훈련 루프
    pd_results = []
    test_accuracy = -987654321
    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels - 1)
            loss.backward()
            optimizer.step()

            # 손실 및 정확도 누적
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels - 1).sum().item()

        # 에포크당 손실 및 정확도 계산
        epoch_loss = running_loss / total
        epoch_accuracy = 100. * correct / total

        results = OrderedDict()
        results['epoch'] = epoch + 1
        results['loss'] = epoch_loss
        results['accuracy'] = epoch_accuracy
        pd_results.append(results)
        df = pd.DataFrame.from_dict(pd_results, orient='columns')

        clear_output(wait=True)
        print(f'Info : {learning_rate, batch_size, drop_out, feature}')
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
        print(df.tail())

        # 에포크마다 테스트 세트에서 평가
        net.eval()
        correct_test = 0
        total_test = 0
        import time
        st = time.time()
        with torch.no_grad():
            for images_test, labels_test in test_loader:
                images_test, labels_test = images_test.to(device), labels_test.to(device)
                outputs_test = net(images_test)
                _, predicted_test = torch.max(outputs_test.data, 1)
                total_test += labels_test.size(0)
                correct_test += (predicted_test == labels_test - 1).sum().item()
        _test_accuracy = 100. * correct_test / total_test
        print(f"Test Accuracy: {_test_accuracy:.2f}%")
        print(f'time cost : {time.time()-st} sec')

        print(f"Max acc test : {test_accuracy if test_accuracy > _test_accuracy else _test_accuracy:.2f}%")
        if _test_accuracy > test_accuracy:
            test_accuracy = _test_accuracy
            torch.save({
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, "emnist_4layer_ver7.pth")




In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from collections import OrderedDict
from IPython.display import clear_output


learning_rate = 0.001
batch_size = 128
num_classes = 26
epochs = 30
Drp = 0.5


train_set = torchvision.datasets.EMNIST(
    root = './data/EMNIST',
    split = 'letters',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor() # 데이터를 0에서 255까지 있는 값을 0에서 1사이 값으로 변환
    ])
)
test_set = torchvision.datasets.EMNIST(
    root = './data/EMNIST',
    split = 'letters',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor() # 데이터를 0에서 255까지 있는 값을 0에서 1사이 값으로 변환
    ])
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

print(train_set, test_set)



if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')


#####
# 신경망 모델 정
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Layer 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),  # Batch Normalization
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 28x28 -> 14x14
        )

        # Layer 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),  # Batch Normalization
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 14x14 -> 7x7
        )

        # Layer 3
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),  # Batch Normalization
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 7x7 -> 3x3
        )

        # Layer 4
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),  # Batch Normalization
            nn.ReLU()
            # No pooling, keeps 3x3 spatial dimension
        )

        # Dropout and Fully Connected Layers
        self.dropout = nn.Dropout(p=Drp)
        self.fc1 = nn.Linear(in_features=128 * 3 * 3, out_features=1000)
        self.fc2 = nn.Linear(in_features=1000, out_features=26)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.reshape(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
#####


# 모델 초기화, 손실 함수 및 최적화 기법 설정
net = NeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)


# 훈련 루프
pd_results = []

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        out = net(images)
        loss = criterion(out, labels-1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total = labels.size(0)
        preds = torch.max(out.data, 1)[1]
        correct = (preds==labels-1).sum().item()

        if (i+1)%200==0:
            results = OrderedDict()
            results['epoch'] = epoch+1
            results['idx'] = i+1
            results['loss'] = loss.item()
            results['accuracy'] = 100.*correct/total
            pd_results.append(results)
            df = pd.DataFrame.from_dict(pd_results, orient='columns')

            clear_output(wait=True)
            # display(df)
            print(df)






     epoch  idx      loss  accuracy
0        1  200  0.699922  77.34375
1        1  400  0.514354  84.37500
2        1  600  0.248082  90.62500
3        1  800  0.458122  89.06250
4        2  200  0.398733  87.50000
..     ...  ...       ...       ...
115     29  800  0.210001  96.87500
116     30  200  0.084324  96.87500
117     30  400  0.123423  94.53125
118     30  600  0.049325  99.21875
119     30  800  0.202616  93.75000

[120 rows x 4 columns]


NameError: name 'model' is not defined

In [4]:
net = NeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)

torch.save({
    'model_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, "emnist_4layer_ver6.pth")


In [5]:
# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# 옵티마이저의 state_dict 출력
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
layer1.0.weight 	 torch.Size([16, 1, 3, 3])
layer1.0.bias 	 torch.Size([16])
layer1.1.weight 	 torch.Size([16])
layer1.1.bias 	 torch.Size([16])
layer1.1.running_mean 	 torch.Size([16])
layer1.1.running_var 	 torch.Size([16])
layer1.1.num_batches_tracked 	 torch.Size([])
layer2.0.weight 	 torch.Size([32, 16, 3, 3])
layer2.0.bias 	 torch.Size([32])
layer2.1.weight 	 torch.Size([32])
layer2.1.bias 	 torch.Size([32])
layer2.1.running_mean 	 torch.Size([32])
layer2.1.running_var 	 torch.Size([32])
layer2.1.num_batches_tracked 	 torch.Size([])
layer3.0.weight 	 torch.Size([64, 32, 3, 3])
layer3.0.bias 	 torch.Size([64])
layer3.1.weight 	 torch.Size([64])
layer3.1.bias 	 torch.Size([64])
layer3.1.running_mean 	 torch.Size([64])
layer3.1.running_var 	 torch.Size([64])
layer3.1.num_batches_tracked 	 torch.Size([])
layer4.0.weight 	 torch.Size([128, 64, 3, 3])
layer4.0.bias 	 torch.Size([128])
layer4.1.weight 	 torch.Size([128])
layer4.1.bias 	 torch.Size([128])
layer4.1.run

In [ ]:
model = torch.load(emnist_4layer_ver6.pth)
model.eval()